#### Blood Transfusion


This data source is originaly from Prof. I-Cheng Yeh, Department of Information Management, Chung-Hua University. To demonstrate the RFMTC marketing model (a modified version of RFM), this study adopted the donor database of Blood Transfusion Service Center in Hsin-Chu City in Taiwan. The service center passes their blood transfusion service bus to one university in Hsin-Chu City to gather blood donated about every three months. To build an FRMTC model, we selected 748 donors at random from the donor database.

We want to predict whether or not a donor will give blood the next time the bus comes to campus.

In [26]:
import pandas as pd
# Import train_test_split method
from sklearn.model_selection import train_test_split
# Import TPOTClassifier and roc_auc_score
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score
# Importing modules
from sklearn import linear_model

In [27]:
df = pd.read_csv(r'C:\Python\transfusion.csv')

In [28]:
df.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


This dataset contains RFMTC model which is a variation of RFM (Recency, Frequency and Monetary). This model is commonly used in marketing for identifying best customers. In this case, te blood donors. 

These 748 donor data, each one included :
- R (Recency - months since last donation),
- F (Frequency - total number of donation),
- Monetary - total blood donated in c.c.),
- Time - months since first donation), and
- Binary variable representing whether he/she donated blood in March 2007 (1 stand for donating blood; 0 stands for not donating blood). 

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [30]:
#Rename 'whether he/she donated blood in March 2007' to 'target' column

df.rename(columns={'whether he/she donated blood in March 2007':'target'}, inplace=True)
df.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


We want to predict whether or not the same donor will give blood the next time the vehicle comes to campus. The model for this is a binary classifier, meaning that there are only 2 possible outcomes:

    0 - the donor will not give blood
    1 - the donor will give blood
    
Target incidence is defined as the number of cases of each individual target value in a dataset. That is, how many 0s in the target column compared to how many 1s? Target incidence gives us an idea of how balanced (or imbalanced) is our dataset.

The data contains 748 donors at random from the donor database

In [31]:
#print target incidence proportions
df.target.value_counts(normalize=True).round(3)

0    0.762
1    0.238
Name: target, dtype: float64

#### Splitting Splitting transfusion into train and test datasets
We'll now use train_test_split() method to split transfusion DataFrame.

Target incidence informed us that in our dataset 0s appear 76% of the time. We want to keep the same structure in train and test datasets

In [32]:
X = df.drop('target', axis=1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42,stratify=y)
X_train.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26
116,2,7,1750,46
661,16,2,500,16
154,2,1,250,2


#### Selecting model using TPOT

TPOT(Tree-based Pipeline Optimization) is a Python Automated Machine Learning tool that optimizes machine learning pipelines using genetic programming. TPOT will automatically explore hundreds of possible pipelines to find the best one for our dataset. Note, the outcome of this search will be a scikit-learn pipeline, meaning it will include any pre-processing steps as well as the model. We are using TPOT to help us explore and optimize further.

In [33]:
# Instantiate TPOTClassifier
tpot = TPOTClassifier(
    generations=6,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

# AUC score for tpot model
tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

# Print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

Optimization Progress:   0%|          | 0/140 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7422459184429089

Generation 2 - Current best internal CV score: 0.7422459184429089

Generation 3 - Current best internal CV score: 0.7422459184429089

Generation 4 - Current best internal CV score: 0.7422459184429089

Generation 5 - Current best internal CV score: 0.7456308339276876

Generation 6 - Current best internal CV score: 0.7464101394881147

Best pipeline: LogisticRegression(CombineDFs(Normalizer(StandardScaler(input_matrix), norm=l2), StandardScaler(input_matrix)), C=25.0, dual=False, penalty=l2)

AUC score: 0.7920

Best pipeline steps:
1. FeatureUnion(transformer_list=[('pipeline',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('normalizer', Normalizer())])),
                               ('standardscaler', StandardScaler())])
2. LogisticRegression(C=25.0, random_state=42)


#### Checking the Variance


In [34]:
X_train.var().round(3)

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64

#### Log Normalization

Monetary (c.c. blood) variance is very high so we need to log transform it.

In [35]:
import numpy as np

X_train_normed, X_test_normed = X_train.copy(), X_test.copy()

# Specify which column to normalize
col_to_normalize = 'Monetary (c.c. blood)'

# Log normalization
for df_ in [X_train_normed, X_test_normed]:
    # Add log normalized column
    df_['monetary_log'] = np.log(df_[col_to_normalize])
    # Drop the original column
    df_.drop(columns=col_to_normalize, inplace=True)

# Check the variance for X_train_normed
X_train_normed.var().round(3)

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64

In [36]:
# Instantiate LogisticRegression
logreg = linear_model.LogisticRegression(
    solver='liblinear',
    random_state=42
)

# Train the model
logreg.fit(X_train_normed, y_train)

# AUC score for tpot model
logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


AUC score: 0.7891


After selecting model using TPOT we get AUC score 0.7850. This is better than target success rate 76%. Then we do log normalized to our training data and the AUC score is 0.7891.
The AUC score imporved by 0.5% 